<h1><center> Seaside Building Optimization </center></h1>
This code performs a multi-hazard damage analysis and optimization of building mitigation options for Seaside, Oregon.


This notebook consists of the following sections:
+ [Importing modules and defining recurrence interval](#input)
+ [Multi-Hazard damage analysis](#dmg_analysis)
  + [Direct economic loss computations](#econ_loss)
  + [Repair time estimates](#repair)
  + [Population dislocation estimates](#disloc)
+ [Optimization](#optimization)
  + [Data aggregation](#agg-opt)
  + [Optimal solution search](#optimal_search)
+ [Plotting Results](#plotting)


The mitigation options that are considered at each parcel are:

| Mitigation option | Description |
| --- | --- |
| 0 | Do nothing / Status Quo | 
| 1 | Retrofit structure to high code |
| 2 | Relocate structure outside the inundation zone | 
| 3 | Decrease repair time | 

*Notebook developed by [Tarun Adluri](https://www.linkedin.com/in/tarunadluri) and [Dylan Sanderson](https://github.com/22dylan)

<a id='input'></a>
***
### Importing modules and defining recurrence interval


In [1]:
import os
import pandas as pd
import geopandas as gpd
import contextily as ctx
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"]=20,20

from pyincore import IncoreClient, Dataset, FragilityService, MappingSet, DataService
from pyincore.analyses.buildingdamage import BuildingDamage
from pyincore.analyses.cumulativebuildingdamage import CumulativeBuildingDamage
from pyincore.analyses.populationdislocation import PopulationDislocation, PopulationDislocationUtil
from pyincore.analyses.housingunitallocation import HousingUnitAllocation
client = IncoreClient()

Enter username: sanderdy
Enter password: ········
Connection successful to IN-CORE services. pyIncore version detected: 0.9.0


#### Specifying recurrence interval to consider

| Recurrence Intervals Options |
| --- | 
| 100 |  
| 250 | 
| 500 |  
| 1,000 |
| 2,500 |
| 5,000 |
| 10,000 |

In [2]:
event = int(input("Select hazard return period in years (e.g. 500): "))

Select hazard return period in years (e.g. 500): 500


##### Specifiying pyIncore hazard information

In [3]:
earthquake_hazard_dict = {100: "5dfa4058b9219c934b64d495", 
                          250: "5dfa41aab9219c934b64d4b2",
                          500: "5dfa4300b9219c934b64d4d0",
                          1000: "5dfa3e36b9219c934b64c231",
                          2500: "5dfa4417b9219c934b64d4d3", 
                          5000: "5dfbca0cb9219c101fd8a58d",
                         10000: "5dfa51bfb9219c934b68e6c2"}

tsunami_hazard_dict = {100: "5bc9e25ef7b08533c7e610dc", 
                      250: "5df910abb9219cd00cf5f0a5",
                      500: "5df90e07b9219cd00ce971e7",
                      1000: "5df90137b9219cd00cb774ec",
                      2500: "5df90761b9219cd00ccff258",
                      5000: "5df90871b9219cd00ccff273",
                      10000: "5d27b986b9219c3c55ad37d0"}

# creating path to damage output
path_to_dmg = os.path.join(os.getcwd(), 'damage_results')
if not os.path.exists(path_to_dmg):
    os.makedirs(path_to_dmg)

# creating path to dislocation output
path_to_dislocation = os.path.join(os.getcwd(), 'dislocation_results')   
if not os.path.exists(path_to_dislocation):
    os.makedirs(path_to_dislocation)

<a id='dmg_analysis'></a>
***
<h2><center> Multi-hazard damage analysis </center><h2>

##### Earthquake building damage

In [4]:
# initializing building damage and fragility service
bldg_dmg = BuildingDamage(client)   
fragility_service = FragilityService(client)

# defining building dataset (GIS point layer)
bldg_dataset_id = "5df40388b9219c06cf8b0c80"
bldg_dmg.load_remote_input_dataset("buildings", bldg_dataset_id)

retrofit_mapping_ids = ["5d2789dbb9219c3c553c7977", 
                        "5e99d145f2935b00011900a4"]

for retrofit_i in range(len(retrofit_mapping_ids)):
    # specifiying mapping id from fragilites to building types
    mapping_id = retrofit_mapping_ids[retrofit_i]
    mapping_set = MappingSet(fragility_service.get_mapping(mapping_id))
    bldg_dmg.set_input_dataset('dfr3_mapping_set', mapping_set)

    bldg_dmg.set_parameter("hazard_type", "earthquake")
    bldg_dmg.set_parameter("num_cpu", 4)

    result_name = os.path.join(path_to_dmg, 'buildings_eq_{}yr_opt{}' .format(event, retrofit_i))
    hazard_id = earthquake_hazard_dict[event]
    bldg_dmg.set_parameter("hazard_id", hazard_id)
    bldg_dmg.set_parameter("result_name", result_name)

    bldg_dmg.run_analysis()

Dataset already exists locally. Reading from local cached zip.
Unzipped folder found in the local cache. Reading from it...


##### Tsunami building damage

In [5]:
# initializing building damage and fragility service
bldg_dmg = BuildingDamage(client)   
fragility_service = FragilityService(client)

# defining building dataset (GIS point layer)
bldg_dataset_id = "5df40388b9219c06cf8b0c80"
bldg_dmg.load_remote_input_dataset("buildings", bldg_dataset_id)

# --- running through tsunami building damage with status quo options
# specifiying mapping id from fragilites to building types

mapping_id = "5d279bb9b9219c3c553c7fba"
mapping_set = MappingSet(fragility_service.get_mapping(mapping_id))
bldg_dmg.set_input_dataset('dfr3_mapping_set', mapping_set)

bldg_dmg.set_parameter("hazard_type", "tsunami")
bldg_dmg.set_parameter("num_cpu", 4)

result_name = os.path.join(path_to_dmg, 'buildings_tsu_{}yr_opt0' .format(event))
hazard_id = tsunami_hazard_dict[event]
bldg_dmg.set_parameter("hazard_id", hazard_id)
bldg_dmg.set_parameter("result_name", result_name)

bldg_dmg.run_analysis()

Dataset already exists locally. Reading from local cached zip.
Unzipped folder found in the local cache. Reading from it...


True

Now, creating output with relocated structures (i.e. tsunami damage is None)

In [6]:
tsu_data = os.path.join(path_to_dmg, 'buildings_tsu_{}yr_opt0.csv' .format(event))
df = pd.read_csv(tsu_data)
df['DS_0'] = 1
df['DS_1'] = 0
df['DS_2'] = 0
df['DS_3'] = 0
df['LS_0'] = 0
df['LS_1'] = 0
df['LS_2'] = 0

path_out = os.path.join(path_to_dmg, 'buildings_tsu_{}yr_opt1.csv' .format(event))
df.to_csv(path_out, index=False)

##### Multi-hazard damage analysis

| Mitigation option | Description | Earthquake | Tsunami |
| --- | --- | --- | --- |
| 0 | Do nothing / Status Quo | Status Quo | Status Quo | 
| 1 | Retrofit structure to high code | Opt1 | Status Quo |
| 2 | Relocate structure outside the inundation zone | Status Quo | Opt1 |
| 3 | Decrease repair time | Status Quo | Status Quo |

In [7]:
cumulative_scenarios = [
    ['buildings_eq_{}yr_opt0.csv' .format(event), 'buildings_tsu_{}yr_opt0.csv' .format(event)],
    ['buildings_eq_{}yr_opt1.csv' .format(event), 'buildings_tsu_{}yr_opt0.csv' .format(event)],
    ['buildings_eq_{}yr_opt0.csv' .format(event), 'buildings_tsu_{}yr_opt1.csv' .format(event)],
    ['buildings_eq_{}yr_opt0.csv' .format(event), 'buildings_tsu_{}yr_opt0.csv' .format(event)]
]


cumulative_bldg_dmg = CumulativeBuildingDamage(client)

for i in range(len(cumulative_scenarios)):
    pth_to_eq = cumulative_scenarios[i][0]
    pth_to_tsu = cumulative_scenarios[i][1]
    pth_to_eq = os.path.join(path_to_dmg, pth_to_eq)
    pth_to_tsu = os.path.join(path_to_dmg, pth_to_tsu)
    
    # --- running pyIncore
    cumulative_bldg_dmg.set_parameter("num_cpu", 1)

    # loading datasets from CSV files into pyincore
    eq_damage_dataset = Dataset.from_file(pth_to_eq, "ergo:buildingDamageVer5")
    tsu_damage_dataset = Dataset.from_file(pth_to_tsu, "ergo:buildingDamageVer5")

    cumulative_bldg_dmg.set_input_dataset("eq_bldg_dmg", eq_damage_dataset)
    cumulative_bldg_dmg.set_input_dataset("tsunami_bldg_dmg", tsu_damage_dataset)

    # defining path to output 
    result_name = os.path.join('buildings_cumulative_{}yr_opt{}' .format(event, i))
    result_name = os.path.join(path_to_dmg, result_name)
    
    cumulative_bldg_dmg.set_parameter("result_name", result_name)

    # running analysis
    cumulative_bldg_dmg.run_analysis()
    

##### Removing individual hazard results
We only care about the multi-hazard case and don't wont too many files hanging around

In [8]:
if os.path.isdir(path_to_dmg):
    for file in os.listdir(path_to_dmg):
        if not "cumulative" in file:
            file = os.path.join(path_to_dmg, file)
            os.remove(file)

<a id='econ_loss'></a>
***
### Direct economic loss computations
Computing economic losses based on expected damage state and 

In [9]:
dmg_rto = [0.005, 0.155, 0.55, 0.90]  # from MAEVIS documentation (damage ratio)

# reading in rmv values of each building
bldg_dataset_id = "5df40388b9219c06cf8b0c80"
data_service = DataService(client)
dataset = Dataset.from_data_service(bldg_dataset_id, data_service)
rd = dataset.get_inventory_reader()


struct_typ, guid, rmv, year_built = [], [], [], []
for row in rd:
    guid.append(row['properties']['guid'])
    rmv.append(row['properties']['rmv_improv'])
    year_built.append(row['properties']['year_built'])
    struct_typ.append(row['properties']['struct_typ'])
df = pd.DataFrame({'guid': guid, 'rmv': rmv, 'year_built': year_built,'struct_typ':struct_typ})
df.set_index('guid', inplace=True)

DS_keys = ['DS_0', 'DS_1', 'DS_2', 'DS_3']
for file in os.listdir(path_to_dmg):
    
    # reading in the results
    file = os.path.join(path_to_dmg, file)
    df_temp = pd.read_csv(file)
    df_temp.set_index('guid', inplace=True)
    
    # adding real market value column to results if it's not already there
    if not 'rmv' in list(df_temp.columns):
        df_temp = pd.merge(df_temp, df, left_index=True, right_index=True)
    
    # computing expected direct economic losses if they do not exist
    if not 'econ_loss' in list(df_temp.columns):
        df_temp['econ_loss'] = (df_temp[DS_keys]*dmg_rto).sum(axis=1)*df_temp['rmv']      
    
    # computing expected direct economic losses if they do not exist
    if not 'year_built' in list(df_temp.columns):
        df_temp['year_built'] = df['year_built']
        df_temp['year_built'].replace([0,1,200,2203], 1960)
        
    # writing back to output
    df_temp.to_csv(file)


Dataset already exists locally. Reading from local cached zip.
Unzipped folder found in the local cache. Reading from it...


<a id='repair'></a>
***
### Repair time estimates
+ Building repair time information is taken from [HAZUS MH 2.1 (Earthquake Model)](https://www.fema.gov/sites/default/files/2020-09/fema_hazus_earthquake-model_technical-manual_2.1.pdf#page=613)  Section 15.2.4.
+ HAZUS provides the median values for repair times for each damage state and building type.
+ The repair time formulation follows [Kameshwar et al. (2019)](https://www.sciencedirect.com/science/article/pii/S0951832018315163?casa_token=CPiMJq2o8zAAAAAA:Hjr5X2tu2MWfcEG57JVwAMrn9QgiInDG_eoPsUAQXdZJ7VgaI3UyXVqILpD92IPTVG50R5MsaVA), in which only one set of repair time estimates are used for both earthquake and tsunami hazards and across building types.
+ Kameshwar et al. (2019) assume that a lognormal restoration model is used for buildings and that the logarithmic dispersion is 0.5.
+ Because Monte-Carlo simulation is not implemented for the optimization algorithm, the median values provided by HAZUS that are used to parameterize the lognormal restoration functions are converted to [mean values](https://www.itl.nist.gov/div898/handbook/apr/section1/apr164.htm)


| Damage State | Median | Dispersion |
| --- | --- | --- |
| Insignificant/None | 0.5 | 0.5 |
| Moderate | 60 | 0.5 |
| Heavy | 360 | 0.5 |
| Complete | 720 | 0.5 |

The repair time is thus computed as <br>
$$Repair = \sum_{ds} P_{ds} \mu_{r,ds}$$
With 
+ $P_{ds}$: probability of being in damage state $ds$
+ $\mu_{r,ds}$: mean repair time associated with damage state $ds$

In [10]:
# median repair times for DS: None, slight, moderate, extensive, and complete
med = np.array([0.5, 60, 360, 720])
beta = np.array([0.5, 0.5, 0.5, 0.5])
DSs = ['DS_0', 'DS_1', 'DS_2', 'DS_3']

# converting median/beta to mean values
mean_repair_time = med*np.exp((beta**2)/2)

In [11]:
DS_keys = ['DS_0', 'DS_1', 'DS_2', 'DS_3']
cnt = 0
for file in os.listdir(path_to_dmg):
    
    # if mitigation option 3, decrease repair time by half
    if cnt == 3:
        mean_repair_time = 0.5*mean_repair_time

    # reading in the results
    file = os.path.join(path_to_dmg, file)
    df_temp = pd.read_csv(file)
    df_temp.set_index('guid', inplace=True)
    
    # computing repair times if they do not exist
    if not 'repair' in list(df_temp.columns):
        df_temp['repair'] = (df_temp[DS_keys]*mean_repair_time).sum(axis=1)      

    df_temp.to_csv(file)
    cnt += 1


## Initial Interdependent Community Description - Seaside, OR

Explore building inventory and social systems. Specifically look at how the building inventory connects with the housing unit inventory using the housing unit allocation.
The housing unit allocation method will provide detail demographic characteristics for the community allocated to each structure.

In [12]:
# Seaside, OR Housing unit inventory, IN-CORE_1bv6_SetupSeaside_FourInventories_2019-08-02_HUinventory.csv
housing_unit_inv = "5d543087b9219c0689b98234"

# Seaside, OR Address point inventory, IN-CORE_1bv6_SetupSeaside_FourInventories_2019-08-02_addresspointinventory.csv
address_point_inv = "5d542fefb9219c0689b981fb"

# Seaside, OR Building inventory, IN-CORE_1bv6_SetupSeaside_FourInventories_2019-08-02_buildinginventory.csv
building_inv = "5df40388b9219c06cf8b0c80"


In [13]:
data_services = DataService(client)

In [17]:
population = pd.read_csv("Seaside_addresspointinventory.csv")
population = population.drop_duplicates(subset=['strctid'])
population.to_csv("seaside_population.csv")
population

,blockid,addrptid,strctid,x,y,huestimate,residential
0,410079511001025,41007000000020005S001001A,41007000000020005S,-123.900452,46.010494,1,1
1,410079511003005,41007000000020009S001001A,41007000000020009S,-123.932060,45.979836,1,1
2,410079507002020,41007020802001001S001001A,41007020802001001S,-123.918625,46.017567,1,1
3,410079507002040,41007020853008001S001001A,41007020853008001S,-123.913643,46.017326,1,1
4,410079507002040,41007020853008002S001001A,41007020853008002S,-123.913643,46.017326,1,1
...,...,...,...,...,...,...,...
5983,410079511003077,41007026497001001S001001A,41007026497001001S,-123.942680,45.972363,1,1
5984,410079511003076,41007026498001001S001001A,41007026498001001S,-123.943321,45.972286,1,1
5985,410079511003002,41007026500001001S001001A,41007026500001001S,-123.941330,45.972206,1,1
5986,410079511003077,41007026501001001S001001A,41007026501001001S,-123.942200,45.972202,1,1


In [18]:
address = Dataset.from_file("seaside_population.csv",data_type="incore:addressPoints")
address

## Run Housing Unit Allocation 
https://github.com/IN-CORE/incore-docs/blob/master/notebooks/housingunitallocation.ipynb

Rosenheim, Nathanael, Roberto Guidotti, Paolo Gardoni & Walter Gillis Peacock. (2019). Integration of detailed household and housing unit characteristic data with critical infrastructure for post-hazard resilience modeling. Sustainable and Resilient Infrastructure. doi.org/10.1080/23789689.2019.1681821

In [ ]:
# Create housing allocation 
hua = HousingUnitAllocation(client)

# Load input dataset
hua.load_remote_input_dataset("housing_unit_inventory", housing_unit_inv)
hua.load_remote_input_dataset("address_point_inventory", address_point_inv)
#hua.set_input_dataset("address_point_inventory", address)
hua.load_remote_input_dataset("buildings", building_inv)

# Specify the result name
result_name = "IN-CORE_1bv6_housingunitallocation"

seed = 1238
iterations = 1

# Set analysis parameters
hua.set_parameter("result_name", result_name)
hua.set_parameter("seed", seed)
hua.set_parameter("iterations", iterations)

In [ ]:
# Run Housing unit allocation analysis
hua.run_analysis()

In [ ]:
# Retrieve result dataset
hua_result = hua.get_output_dataset("result")

# Convert dataset to Pandas DataFrame
hua_df = hua_result.get_dataframe_from_csv(low_memory=False)

# Display top 5 rows of output data
hua_df[['guid','numprec','incomegroup']].head()

In [ ]:
#hua_df = pd.read_csv("IN-CORE_1bv6_housingunitallocation_1238.csv")

## Found Issue - to be fixed
The building inventory for Seaside has damage state information - a new version of the file needs to be made without the damage data.

### Seaside, OR Building inventory, IN-CORE_1bv6_SetupSeaside_FourInventories_2019-08-02_buildinginventory.csv
building_inv = "5d5433edb9219c0689b98344"

In [ ]:
hua_df = hua_df.drop(columns= ['insignific','moderate','heavy','complete'])
hua_df.head()

In [ ]:
hua_df = hua_df.loc[hua_df['aphumerge'] == 'both']

In [ ]:
hua_df['Race Ethnicity'] = "0 Vacant HU No Race Ethnicity Data"
hua_df['Race Ethnicity'].notes = "Identify Race and Ethnicity Housing Unit Characteristics."

hua_df.loc[(hua_df['race'] == 1) & (hua_df['hispan'] == 0),'Race Ethnicity'] = "1 White alone, Not Hispanic"
hua_df.loc[(hua_df['race'] == 2) & (hua_df['hispan'] == 0),'Race Ethnicity'] = "2 Black alone, Not Hispanic"
hua_df.loc[(hua_df['race'].isin([3,4,5,6,7])) & (hua_df['hispan'] == 0),'Race Ethnicity'] = "3 Other Race, Not Hispanic"
hua_df.loc[(hua_df['hispan'] == 1),'Race Ethnicity'] = "4 Any Race, Hispanic"
hua_df.loc[(hua_df['gqtype'] >= 1),'Race Ethnicity'] = "5 Group Quarters no Race Ethnicity Data"

# Check new variable
table_title = "Confirm housing unit characteristic by Race and Ethnicity."
pd.crosstab(hua_df['Race Ethnicity'], hua_df['race'], 
            margins=True, margins_name="Total").style.set_caption(table_title)

In [ ]:
# Check new variable
table_title = "Confirm housing unit characteristic by Race and Ethnicity."
pd.crosstab(hua_df['Race Ethnicity'], hua_df['hispan'], 
            margins=True, margins_name="Total").style.set_caption(table_title)

In [ ]:
table_title = "Table 1. Housing Unit Characteristics by Race and Ethnicity"
table1 = pd.pivot_table(hua_df, values='numprec', index=['Race Ethnicity'],
                              margins = True, margins_name = 'Total',
                              aggfunc=[len, np.sum], 
                              fill_value=0).reset_index().rename(
                                                            columns={'len': 'Housing Unit',
                                                                     'sum' : 'Population',
                                                                     'numprec': 'Count'})

varformat = {('Housing Unit','Count'): "{:,}", ('Population','Count'): "{:,}"}

In [ ]:
table1.style.set_caption(table_title).format(varformat).set_table_styles([
    dict(selector='th', props=[('text-align', 'center')]),])

## Validate the Housing Unit Allocation has worked
Notice that the population count totals for the community should match (pretty closely) data collected for the 2010 Decennial Census.
This can be confirmed by going to data.census.gov

https://data.census.gov/cedsci/table?q=DECENNIALPL2010.P1&g=1600000US4165950&tid=DECENNIALSF12010.P1

Differences in the housing unit allocation and the Census count may be due to differences between political boundaries and the building inventory. See Rosenheim et al 2019 for more details.

The housing unit allocation results will become the input for the dislocation model.

In [ ]:
# Save cleaned HUA file as CSV
hua_df.to_csv('IN-CORE_1cv1_housingunitallocation_1238.csv')

In [ ]:
# Create population dislocatin 
pop_dis = PopulationDislocation(client)

In [ ]:
# Seaside, OR Housing unit allocation, performed at start of notebook
housing_unit_alloc = Dataset.from_file('IN-CORE_1cv1_housingunitallocation_1238.csv','incore:housingUnitAllocation')

# Seaside, OR "IN-CORE_1bv6_SetupSeaside_FourInventories_2019-08-02_bgdata.csv"
bg_data = "5d542bd8b9219c0689b90408"

# Value loss parameters, "IN-CORE_value_loss_bai09.csv"
value_loss = "5dfd1069fc33d500081555d8"

# Load input dataset
pop_dis.set_input_dataset("housing_unit_allocation", housing_unit_alloc)
pop_dis.load_remote_input_dataset("block_group_data", bg_data)
pop_dis.load_remote_input_dataset("value_poss_param", value_loss)

In [ ]:
# Create population dislocatin 
pop_dis = PopulationDislocation(client)

# Load input dataset
bldg_dmg_result = Dataset.from_file("buildings_cumulative_500yr_opt0.csv", data_type='ergo:buildingDamageVer5')
pop_dis.set_input_dataset("building_dmg", bldg_dmg_result)
pop_dis.set_input_dataset("housing_unit_allocation", housing_unit_alloc)
pop_dis.load_remote_input_dataset("block_group_data", bg_data)
pop_dis.load_remote_input_dataset("value_poss_param", value_loss)

# Specify the result name
result_name = "IN-CORE_1bv6_population_dislocation"

seed = 1111

# Set analysis parameters
pop_dis.set_parameter("result_name", result_name)
pop_dis.set_parameter("seed", seed)

In [ ]:
pop_dis.run_analysis()

<a id='disloc'></a>
***
### Population dislocation estimates

In [ ]:
pop_dis = PopulationDislocation(client)
seed = 1111
cnt = 0
for file in os.listdir(path_to_dmg):

    #Reading in local building damage files
    file = os.path.join(path_to_dmg, file)
    dmg_data = Dataset.from_file(file, data_type='ergo:buildingDamageVer5')
    pop_dis.set_input_dataset("building_dmg", dmg_data)

    # loading remote population dislocation data 
    pop_dis.load_remote_input_dataset("housing_unit_allocation", "5d543b06b9219c0689b987af")
    pop_dis.load_remote_input_dataset("block_group_data", "5d542bd8b9219c0689b90408")
    pop_dis.load_remote_input_dataset("value_poss_param", "5dfd1069fc33d500081555d8") 

    # setting population dislocation run information
    result_name = "housing-dislocation-result-opt{}".format(cnt)
    result_name = os.path.join(path_to_dislocation, result_name)
    pop_dis.set_parameter("result_name", result_name)
    pop_dis.set_parameter("seed", seed)
    
    # running population dislocation
    pop_dis.run_analysis()
    
    # reading 
    df = pd.read_csv('{}.csv' .format(result_name))
    df.dislocated = df.dislocated.astype("int32")
    df.dropna(inplace=True)
    df = df.drop_duplicates(subset=['guid'])
    df.to_csv(result_name+".csv", index=False)
    cnt += 1

In [ ]:
dmg_files = os.listdir(path_to_dmg)
dslc_files = os.listdir(path_to_dislocation)

for i in range(4):
    dmg_file = os.path.join(path_to_dmg, dmg_files[i])
    dmg_df = pd.read_csv(dmg_file)
    dmg_df.set_index('guid', inplace=True)

    if not 'dislocated' in list(dmg_df.columns):
        dslc_file = os.path.join(path_to_dislocation, dslc_files[i])
        dslc_df = pd.read_csv(dslc_file)
        dslc_df.set_index('guid', inplace=True)    
        dmg_df = pd.merge(dmg_df, dslc_df['dislocated'], how='left', left_index=True, right_index=True)

        dmg_df.to_csv(dmg_file, index=True)

##### Removing population dislocation results
This data has been compiled into the damage anlaysis files

In [ ]:
if os.path.isdir(path_to_dislocation):
    for file in os.listdir(path_to_dislocation):
        file = os.path.join(path_to_dislocation, file)
        os.remove(file)
    os.rmdir(path_to_dislocation)

*** 
<a id='optimization'></a>
<h2><center> Optimization </center><h2>

<a id='agg-opt'></a>
***
### Data aggregation

In [ ]:
level0 = pd.read_csv("damage_results/buildings_cumulative_{}yr_opt0.csv".format(event))
level0["K"]= 0
level0["b"]=1
level1 = pd.read_csv("damage_results/buildings_cumulative_{}yr_opt1.csv".format(event))
level1["K"]= 1
level1["b"]=0
level1["dislocated"] = level1.dislocated*0.80
level2 = pd.read_csv("damage_results/buildings_cumulative_{}yr_opt2.csv".format(event))
level2["K"]= 2
level2["b"]=0
level2["dislocated"] = level1.dislocated*0.50
level3 = pd.read_csv("damage_results/buildings_cumulative_{}yr_opt3.csv".format(event))
level3["K"]= 3
level3["b"]=0
level3["dislocated"] = level1.dislocated*1
level0.fillna(0, inplace=True)
level1.fillna(0, inplace=True)
level2.fillna(0, inplace=True)
level3.fillna(0, inplace=True)
print(level1[['guid', 'dislocated']].head())

In [ ]:
ss_all_lvls = pd.concat([level0, level1, level2, level3])
ss_all_lvls = ss_all_lvls.groupby(["guid", "struct_typ", "K"]).mean().reset_index()
ss_all_lvls

<a id='optimization'></a>
***
### Optimization Datasets


### Future Metrics

In [ ]:
qt_data = ss_all_lvls.copy()
qt_data.rename(columns = {'guid':'Z'}, inplace = True)
qt_data.rename(columns = {'struct_typ':'S'}, inplace = True)
qt_data.rename(columns = {'dislocated':'d_ijk'}, inplace = True)
qt_data.rename(columns = {'econ_loss':'l'}, inplace = True)
qt_data["Q_t_hat"] = ss_all_lvls.repair/ss_all_lvls.b.sum()
qt_data = qt_data.drop(['LS_0', 'LS_1', 'LS_2', 'DS_0','DS_1', 'DS_2', 'DS_3','year_built'], axis=1)
qt_data.to_csv("seaside_qt_data_{}.csv".format(event))
qt_data

In [ ]:
print("Total Buildings:",qt_data.b.sum())
print("Repair time:",np.sum(qt_data.b*qt_data.Q_t_hat))
print("Buildings Dislocated:",np.sum(qt_data.b*qt_data.d_ijk))
print("Economic Loss:",np.sum(qt_data.b*qt_data.l))

### Strategy Costs

In [ ]:
level0['k1cost'] = level0.rmv*0.20  #30% of rmv for stategy 1
level0['k2cost'] = level0.rmv*1  #100% of rmv for stategy 2
level0['k3cost'] = level0.rmv*0.10  #15% of rmv for stategy 3

In [ ]:
def calculate_Sc(df_lQaddedb,df):
    df_Sc=df_lQaddedb[:]
    df_Sc=df_Sc.loc[df_Sc.index.repeat(4)].reset_index(drop=True)
    df_Sc["K'"]=[0,1,2,3]*len(df_lQaddedb)
    df_Sc.drop(["b","Q_t_hat"],axis=1,inplace=True)
    df_Sc["Sc"]=0
    for index,row in df_Sc.iterrows():
        r=df[(df["guid"]==row["Z"])&(df["struct_typ"]==row["S"])]
        if row["K'"]<row["K"]:
            df_Sc.loc[index,"Sc"]=100000000000000
        if row["K"]==0:
            if row["K'"]==1:
                df_Sc.loc[index,"Sc"]=r["k1cost"].item()
            elif row["K'"]==2:
                df_Sc.loc[index,"Sc"]=r["k2cost"].item()
            elif row["K'"]==3:
                df_Sc.loc[index,"Sc"]=r["k3cost"].item()
#         elif row["K"]==1:
#             if row["K'"]==2:
#                 df_Sc.loc[index,"Sc"]=r["k2cost"].item()
#             elif row["K'"]==3:
#                 df_Sc.loc[index,"Sc"]=r["k3cost"].item()
#         elif row["K"]==2:
#             if row["K'"]==3:
#                 df_Sc.loc[index,"Sc"]=r["k3cost"].item()
    df_Sc.drop( df_Sc[ df_Sc["K'"]<df_Sc["K"] ].index , inplace=True)
    df_Sc = df_Sc[df_Sc["K"]==0]
    return df_Sc

df_sc = calculate_Sc(qt_data, level0)
df_sc

In [ ]:
df_sc.to_csv("seaside_sc_data_{}.csv".format(event))

<a id='plotting'></a>
***
<h2><center> Plotting Results </center><h2>

#### Reading in building polygon dataset and setting up function to filter data

In [ ]:
# reading in building polygon dataset
bldg_dataset_id = "5d927ab2b9219c06ae8d313c"
data_service = DataService(client)
dataset = Dataset.from_data_service(bldg_dataset_id, data_service)
rd = dataset.get_inventory_reader()

# setting up geodataframe
gdf = gpd.GeoDataFrame.from_features([feature for feature in rd], crs="EPSG:3857")
gdf = gdf[['guid', 'geometry']]
gdf.dropna(subset=['guid'], inplace=True)
gdf.drop_duplicates(subset='guid', inplace=True)
gdf.set_index('guid', inplace=True)

def get_gdf_feats(gdf, path_to_run=None, values=None, k=None):
    df = pd.read_csv(path_to_run)
    df.set_index('Z', inplace=True)
    if values != None:
        df = df.loc[df['Values'].isin(values)]
    if k != None:
        df = df.loc[df['K'].isin(k)]
    gdf_new = pd.merge(gdf, df, left_index=True, right_index=True)
    return gdf_new

#### Drawing map

In [ ]:
# --- making figure and map
fig, ax = plt.subplots(1,1)
gdf = gdf.to_crs(epsg=3857)

# plotting all parcels
gdf.plot(ax=ax, color='lightblue')

# getting a subset of parcels
gdf_k0 = get_gdf_feats(gdf, path_to_run='decision_variable_B60_X369.csv', values=[1,2], k=[0])

# drawing the subset of parcels on the map
gdf_k0.plot(ax=ax, color='salmon')


# adding basemap
ctx.add_basemap(ax, zoom=15, crs='EPSG:4326', source=ctx.providers.Stamen.TonerLite)
# ctx.add_basemap(ax, zoom=15, crs='EPSG:4326', source=ctx.providers.OpenStreetMap.Mapnik)
